In [0]:
#| default_exp patching.first_patching

# Patchify Image
> Create first patching of images

In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [2]:
#| export
from cv_tools.core import *
from cv_tools.imports import *
from cv_tools.data_processing.smb_tools import *


In [4]:
#| export
import torch
from torch import nn


In [12]:
img = torch.randn(1, 1, 1152, 1632)
H, W = img.shape[2:]
tile_size = 256
overlap = 32
h_steps = max(1, (H - overlap) // (tile_size - overlap))

In [0]:
#| export
class TileProcessor(nn.Module):
    def __init__(self, tile_size=256, overlap=32):
        super().__init__()
        self.tile_size = tile_size
        self.overlap = overlap
    
    def forward(self, x):
        """Split image into overlapping tiles and return tiles with their positions"""
        B, C, H, W = x.shape
        tile_size = self.tile_size
        overlap = self.overlap
        
        tiles = []
        positions = []
        
        # Calculate steps with overlap
        h_steps = max(1, (H - overlap) // (tile_size - overlap))
        w_steps = max(1, (W - overlap) // (tile_size - overlap))
        
        for i in range(h_steps):
            h_start = min(i * (tile_size - overlap), H - tile_size)
            for j in range(w_steps):
                w_start = min(j * (tile_size - overlap), W - tile_size)
                
                # Extract tile
                tile = x[:, :, h_start:h_start+tile_size, w_start:w_start+tile_size]
                tiles.append(tile)
                
                # Store position
                positions.append([h_start, w_start, h_start+tile_size, w_start+tile_size])
        
        # If we have any space left at the bottom or right, add extra tiles
        if h_steps * (tile_size - overlap) + overlap < H:
            h_start = H - tile_size
            for j in range(w_steps):
                w_start = min(j * (tile_size - overlap), W - tile_size)
                tile = x[:, :, h_start:h_start+tile_size, w_start:w_start+tile_size]
                tiles.append(tile)
                positions.append([h_start, w_start, h_start+tile_size, w_start+tile_size])
        
        if w_steps * (tile_size - overlap) + overlap < W:
            w_start = W - tile_size
            for i in range(h_steps):
                h_start = min(i * (tile_size - overlap), H - tile_size)
                tile = x[:, :, h_start:h_start+tile_size, w_start:w_start+tile_size]
                tiles.append(tile)
                positions.append([h_start, w_start, h_start+tile_size, w_start+tile_size])
        
        # Also add the corner if needed
        if (h_steps * (tile_size - overlap) + overlap < H and 
            w_steps * (tile_size - overlap) + overlap < W):
            h_start = H - tile_size
            w_start = W - tile_size
            tile = x[:, :, h_start:h_start+tile_size, w_start:w_start+tile_size]
            tiles.append(tile)
            positions.append([h_start, w_start, h_start+tile_size, w_start+tile_size])
        
        # Stack tiles into a batch
        tiles = torch.cat(tiles, dim=0)
        positions_tensor = torch.tensor(positions, device=x.device)
        
        return tiles, positions_tensor

In [3]:
#| hide
import nbdev; nbdev.nbdev_export('21_patching.first_patching.ipynb')